### Imports

In [52]:
import os
import datetime
import logging
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr


import ocean_navigation_simulator
from ocean_navigation_simulator.environment.PlatformState import (
    PlatformState,
    SpatialPoint,
)
from ocean_navigation_simulator.reinforcement_learning.missions.MissionGenerator import (
    MissionGenerator,
)
from ocean_navigation_simulator.reinforcement_learning.missions.CachedNavigationProblem import (
    CachedNavigationProblem,
)

from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)


from ocean_navigation_simulator.utils import units
from ocean_navigation_simulator.utils.misc import get_c3, set_arena_loggers

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
c3 = get_c3()

# Set up the experiments

In [54]:
experiment_name = "safety_c3_debug_6"
exp = c3.Experiment.get(experiment_name)
# exp.remove()

In [55]:
exp

### Config

In [56]:
timeout_sim_h = 24 * 5
arena_config = {
    "casadi_cache_dict": {"deg_around_x_t": 2.0, "time_around_x_t": 432000},
    "platform_dict": {
        "battery_cap_in_wh": 400.0,
        "u_max_in_mps": 0.1,
        "motor_efficiency": 1.0,
        "solar_panel_size": 0.5,
        "solar_efficiency": 0.2,
        "drag_factor": 675.0,
        "dt_in_s": 600.0,
    },
    "use_geographic_coordinate_system": True,
    "spatial_boundary": None,
    "ocean_dict": {
        "region": "Region 1",
        "hindcast": {
            "field": "OceanCurrents",
            "source": "hindcast_files",
            "source_settings": {
                "folder": "data/miss_gen_hindcast/",
                "local": False,
                "source": "HYCOM",
                "type": "hindcast",
                "currents": "total",
            },
        },
        "forecast": None,
    },
    "bathymetry_dict": {
        "field": "Bathymetry",
        "source": "gebco",
        "source_settings": {
            "filepath": "ocean_navigation_simulator/package_data/bathymetry_and_garbage/bathymetry_global_res_0.083_0.083_max.nc"
        },
        "casadi_cache_settings": {"deg_around_x_t": 10},
        "use_geographic_coordinate_system": True,
    },
    "garbage_dict": {
        "field": "Garbage",
        "source": "Lebreton",
        "source_settings": {
            "filepath": "ocean_navigation_simulator/package_data/bathymetry_and_garbage/garbage_patch_global_res_0.083_0.083.nc"
        },
        "casadi_cache_settings": {"deg_around_x_t": 10},
        "use_geographic_coordinate_system": True,
    },
    "solar_dict": {"hindcast": None, "forecast": None},
    "seaweed_dict": {"hindcast": None, "forecast": None},
}
config = {
    ##### Target Sampling #####
    # HYCOM HC: lon [-98.0,-76.4000244140625], lat[18.1200008392334,31.92000007629394]
    # Copernicus FC: lon: [-98.0, -76.416664], lat: [18.083334, 30.0]
    # Combined: [-98.0, -76.416664], [18.1200008392334, 30.0]
    "x_range": [-159, -106],
    "y_range": [16, 39],
    # "x_range": [-160, -105],
    # "y_range": [15, 40],
    "t_range": [  # ['2022-08-01T00:00:00+00:00', '2022-08-30T00:00:00+00:00'],
        # Copernicus FC: 2022-04 until today, HYCOM Hindcast: 2021-09 until today
        datetime.datetime(year=2022, month=8, day=16, tzinfo=datetime.timezone.utc),
        datetime.datetime(year=2022, month=8, day=30, tzinfo=datetime.timezone.utc),
    ],
    "problem_timeout_in_h": timeout_sim_h,  # to be considered as feasible
    "problem_target_radius": 0.1,
    ##### HJ Planner #####
    "hj_specific_settings": {
        # 'grid_res' has to be smaller than target_radius to prevent hj_solver errors
        "grid_res": 0.02,  # Note: this is in deg lat, lon (HYCOM Global is 0.083 and Mexico 0.04)
        "direction": "multi-time-reach-back",
        "n_time_vector": 199,  # Note that this is the number of time-intervals, the vector is +1 longer because of init_time
        "accuracy": "high",
        "artificial_dissipation_scheme": "local_local",
        "run_without_x_T": True,
        "progress_bar": False,
        "T_goal_in_seconds": 3600 * timeout_sim_h,  # 7 days here #
    },
    "hj_planner_box": 2.0,
    ##### Start Sampling #####
    "feasible_missions_per_target": 2,
    "feasible_mission_time_in_h": [80, 110],
    "random_missions_per_target": 0,
    "random_min_distance_from_target": 0.5,
    "min_distance_from_hj_frame": 0.5,
    "min_distance_from_land": 0.5,
    "random_min_distance_from_target": 0.5,
    # Safety specific values
    "max_distance_from_land": 1,
    "min_distance_from_garbage": 0.01,
    "max_distance_from_garbage": 1,
    ##### Actions #####
    "plot_batch": False,
    "animate_batch": False,
    "cache_forecast": False,
    "cache_hindcast": False,
    ##### Distance maps #####
    "filepath_distance_map": {
        "bathymetry": "ocean_navigation_simulator/package_data/bathymetry_and_garbage/bathymetry_distance_res_0.083_0.083_max_elevation_-150.nc",
        "garbage": "ocean_navigation_simulator/package_data/bathymetry_and_garbage/garbage_patch_distance_res_0.083_0.083_max.nc",
    },
}
objectiveConfig = {"type": "nav"}

### Create Experiment

Make sure you really want to create a new experiment and you ran `exp = c3.Experiment.get(experiment_name)` before '


In [57]:
# create Experiment
exp = c3.Experiment.createNew(
    name=experiment_name,
    description="safety",
    arenaConfig=arena_config,
    objectiveConfig=objectiveConfig,
    timeout_in_sec=3600 * timeout_sim_h,
)

In [58]:
exp = exp.get("experimentName")

# name.experimentName = "test_matthias"
# name.merge()
exp

c3.Experiment(
 id='safety_c3_debug_6',
 meta=c3.Meta(
        fetchInclude='[experimentName,id,version]',
        fetchType='Experiment'),
 version=1,
 experimentName='safety_c3_debug_6',
 T_horizon_FC_Error_Calculation_in_h=96)

# Set up generate batch job

## Set up and run mission generation

In [59]:
options = c3.GenMissionBatchJobOptionsAndreas(
    **{
        "experiment": exp,
        "missGenBatchConfig": config,
        "base_seed": 1,
        "nBatches": 2,  # each nBatch runs nMissionsGenPerBatch
        "nMissionGenPerBatch": 2,  # nb workers on which we loop,
        #'hardwareProfileId': "appc8m64-w"
    }
)
# total amount of missions: feasible_mission_per_target*nBatches*nMIssionGenPerBatch

#### [Debug locally]


In case you want to debug the Gen Mission c3 stuff locally. Just copy the according code from `GenMissionBatchJob.py`


In [60]:
batch = c3.GenMissionBatchJobBatchAndreas(**{"batch_seed": 42})
options = options

In [61]:
# Set up file paths and download folders
temp_folder = "/tmp/" + str(batch.batch_seed) + "/"
# create the folder if it doesn't exist yet
if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)
log_file_name = str(batch.batch_seed) + ".log"
logging.basicConfig(filename=temp_folder + log_file_name, level=logging.DEBUG)

# change to this if basic setup works test
set_arena_loggers(logging.INFO)
logging.getLogger("MissionGenerator").setLevel(logging.DEBUG)

In [62]:
# get configs for mission generation
arena_config = options.experiment.get("arenaConfig").arenaConfig
# create the MissionGenerator Config
config = options.missGenBatchConfig

# Lots of replacement
filepath = ocean_navigation_simulator.__file__
module_path = os.path.dirname(filepath)

if "filepath_distance_map" in config and config["filepath_distance_map"] is not None:
    for key in config["filepath_distance_map"]:
        config["filepath_distance_map"][key] = config["filepath_distance_map"][key].replace(
            "ocean_navigation_simulator", module_path
        )

data_dicts = ["bathymetry_dict", "garbage_dict"]
for d in data_dicts:
    if d in arena_config and arena_config[d] is not None:
        arena_config[d]["source_settings"]["filepath"] = arena_config[d]["source_settings"][
            "filepath"
        ].replace("ocean_navigation_simulator", module_path)


if "obstacle_dict" in config and config["obstacle_dict"] is not None:
    config["obstacle_dict"]["path_to_obstacle_file"] = config["obstacle_dict"][
        "path_to_obstacle_file"
    ].replace("ocean_navigation_simulator", module_path)

config_to_add = {
    "scenario_config": arena_config,
    "problem_timeout": datetime.timedelta(
        seconds=options.experiment.get("timeout_in_sec").timeout_in_sec
    ),
    "T_goal_in_seconds": options.experiment.get("timeout_in_sec").timeout_in_sec,
}

config = config | config_to_add

In [63]:
config

{'x_range': [-159, -106],
 'y_range': [16, 39],
 't_range': [datetime.datetime(2022, 8, 16, 0, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2022, 8, 30, 0, 0, tzinfo=datetime.timezone.utc)],
 'problem_timeout_in_h': 120,
 'problem_target_radius': 0.1,
 'hj_specific_settings': {'grid_res': 0.02,
  'direction': 'multi-time-reach-back',
  'n_time_vector': 199,
  'accuracy': 'high',
  'artificial_dissipation_scheme': 'local_local',
  'run_without_x_T': True,
  'progress_bar': False,
  'T_goal_in_seconds': 432000},
 'hj_planner_box': 2.0,
 'feasible_missions_per_target': 2,
 'feasible_mission_time_in_h': [80, 110],
 'random_missions_per_target': 0,
 'random_min_distance_from_target': 0.5,
 'min_distance_from_hj_frame': 0.5,
 'min_distance_from_land': 0.5,
 'max_distance_from_land': 1,
 'min_distance_from_garbage': 0.01,
 'max_distance_from_garbage': 1,
 'plot_batch': False,
 'animate_batch': False,
 'cache_forecast': False,
 'cache_hindcast': False,
 'filepath_distance_map': {'bath

In [64]:
results_folder = "/tmp/missions/"
os.makedirs(results_folder, exist_ok=True)
# seed all of them
random.seed(batch.batch_seed)
all_problems = []
for worker in range(options.nMissionGenPerBatch):
    mission_generator = MissionGenerator(
        config=config
        | {
            "seed": int(random.uniform(0, 1000000)),
            "cache_folder": results_folder + str(worker) + "_",
        },
        c3=c3,
    )
    problems, _, _ = mission_generator.cache_batch()
    all_problems.extend(problems)

# Now upload all of them to the mission database!
exp = options.experiment
for idx, problem in enumerate(all_problems):
    try:
        exp.addMission(
            missionConfig=problem.to_c3_mission_config(),
            name="batch_seed_" + str(batch.batch_seed) + "_" + str(idx),
        )
    except BaseException:
        continue

INFO:arena.factory:Downloading 7 files to 'data/miss_gen_hindcast/'.
100%|██████████| 7/7 [00:01<00:00,  3.76it/s]
INFO:arena.factory:Download Hindcast Files: 2022-08-16 11-02-00 until 2022-08-22 13-02-00 (2.09s)
INFO:arena.ocean_field:DataField: Create Hindcast Source (2.2s)
INFO:arena.ocean_field:DataField: Forecast is the same as Hindcast for OceanCurrents.
INFO:arena:Arena: Generate Sources (2.3s)
INFO:arena:Arena: Generate Platform (0.0s)
INFO:arena.factory:ArenaFactory: Creating Arena for None (4.38s)
INFO:arena.platform:Platform: Set Dynamics F_x_next Function (0.0s)
INFO:arena.platform:Platform: Update Casadi + Dynamics (9.6s)
INFO:arena.controller:HJPlannerBase: Planning because of new forecast (Old: None, New: 0)
INFO:arena.controller:HJPlannerBase: Loading new Current Data (2.8s)
INFO:arena.controller:HJPlannerBase: Running multi-time-reach-back


In [ ]:
all_problems

In [ ]:
# all_problems[0]
all_problems[0].extra_info

### Start Jobs

In [ ]:
gen_mission_job = c3.GenMissionBatchJobAndreas(**{"options": options.toJson()}).upsert()
# gen_mission_job = c3.GenMissionBatchJob(**{'options': options.toJson()}).upsert()
# gen_mission_job.setMaxConcurrency(12)
# gen_mission_job.setMaxConcurrencyPerNode(15)
gen_mission_job.start()
gen_mission_job

In [ ]:
# gen_mission_job = c3.GenMissionBatchJobNicolas.get('267cbcfb-374e-496b-9d9b-c1b9a9f960be')
gen_mission_job.status()

In [ ]:
gen_mission_job.computingActions()
# gen_mission_job.cancel()

In [ ]:
# Other Utils for the cluster
c3.Cluster.actionDump()

In [ ]:
len(exp.get("expMissions").expMissions)

In [ ]:
# gen_mission_job.cancel() # to cancel the jobs

In [ ]:
exp.get("expMissions").expMissions[0].get("missionConfig").missionConfig


### Save and load missions from and to experiments instead of generating new ones



#### Experiment to take the missions from


In [ ]:
problems = []
for miss in exp.get("expMissions").expMissions:
    missionConfig = miss.get("missionConfig").missionConfig
    problems.append(CachedNavigationProblem.from_c3_mission_config(missionConfig))

In [ ]:
results_folder = f"{os.getcwd()}/"
df = pd.DataFrame([problem.to_dict() for problem in problems])
df["factory_index"] = 0
df.to_csv(results_folder + "problems.csv")

In [ ]:
len(problems)


##### Make sure you create/select a new experiment where you want to add the retrieved missions to


In [ ]:
exp

In [ ]:
# Some stuff in matthias notebook

In [ ]:
for problem in problems:
    exp.addMission(problem.to_c3_mission_config())

In [ ]:
len(exp.get("expMissions").expMissions)

## Plot all of them on a map and get distribution information

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.set_title("Time: " + datetime.fromtimestamp(time, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC'))

ax.add_feature(cfeature.LAND, zorder=3, edgecolor="black")
ax.add_feature(cfeature.OCEAN, zorder=0)
grid_lines = ax.gridlines(draw_labels=True, zorder=5)
grid_lines.top_labels = False
grid_lines.right_labels = False
ax.axis(xmin=-160, xmax=-105)
ax.axis(ymin=15, ymax=40)
# TODO: adapt Generation runner to use nice plotting
for miss_idx in range(len(df)):
    ax.scatter(
        df.iloc[miss_idx]["x_0_lon"],
        df.iloc[miss_idx]["x_0_lat"],
        c="red",
        marker="o",
        s=6,
        label="starts" if miss_idx == 0 else None,
    )
ax.scatter(df["x_T_lon"], df["x_T_lat"], c="green", marker="x", s=12, label="targets")
ax.legend()
# ax.get_figure().savefig(f"{analysis_folder}starts_and_targets.png")
ax.get_figure().show()

In [ ]:
GenerationRunner.plot_ttr_histogram(results_folder)
GenerationRunner.plot_starts_and_targets(
    results_folder=results_folder,
    pb_filename="problems.csv",
    scenario_config=arena_config,
)

#### Random utils

In [ ]:
status = " && status ==" + '"' + "running" + '"'
all_gen_mission_jobs = c3.GenMissionBatchJobAndreas.fetch(
    spec={
        "include": "[this]",
    }
).objs

In [ ]:
for job in all_gen_mission_jobs:
    if job.get("BatchJobStatus").status().status == "running":
        print(job.get("BatchJobStatus").status().status)
        # job.cancel()
        print(job.get("BatchJobStatus").status().status)

# Add Observer

In [ ]:
# Config for No observer
observer_config = {"observer": None}
# add observer
exp.addObserver(name="NoObserver", observerConfig=observer_config)

# Add Controller

In [ ]:
# Controller Configs
HJMultiTimeConfig = {
    "replan_every_X_seconds": None,
    "replan_on_new_fmrc": True,
    "T_goal_in_seconds": 3600 * 24 * 6,  # 3d, 43200,     # 12h
    "accuracy": "high",
    "artificial_dissipation_scheme": "local_local",
    "ctrl_name": "ocean_navigation_simulator.controllers.hj_planners.HJReach2DPlanner.HJReach2DPlanner",
    "d_max": 0.0,
    "deg_around_xt_xT_box": 1.0,
    "direction": "multi-time-reach-back",
    "grid_res": 0.02,
    "n_time_vector": 200,
    "progress_bar": True,
    "use_geographic_coordinate_system": True,
}
StraightLineConfig = {
    "ctrl_name": "ocean_navigation_simulator.controllers.NaiveController.NaiveController"
}
flockingConfig = {
    "unit": "km",
    "interaction_range": 9,  # km
    "grad_clip_range": 0.1,  # km
}
reactiveConfig = {
    "unit": "m",
    "mix_ttr_and_euclidean": False,
    "delta_3": 8800,  # collision threshold (communication - delta_3)
    "delta_2": 2000,  # safe zone threshold  ]communication - delta_2, communication - delta_3[
    "delta_1": 500,  # small threshold so that if distance > communication_thrsld- delta_1 we try to achieve connectivity
    "communication_thrsld": 9000,
    "k_1": 0.25,
    "k_2": 1,
}

#### Add HJ Naive controller

In [ ]:
MultiAgentCtrlConfig = {
    "ctrl_name": "ocean_navigation_simulator.controllers.MultiAgentPlanner.MultiAgentPlanner",
    "high_level_ctrl": "hj_naive",  # choose from hj_naive, flocking, reactive_control
    "unit": "km",
    "communication_thrsld": 9,
    "hj_specific_settings": HJMultiTimeConfig,
    "flocking_config": flockingConfig,
    "reactive_control_config": reactiveConfig,
}

In [ ]:
exp.experimentName = exp.get("experiment_name")
# add controller
exp.addController(name="hj_naive", ctrlConfig=MultiAgentCtrlConfig)

#### Add Reactive Controller

In [ ]:
MultiAgentCtrlConfig = {
    "ctrl_name": "ocean_navigation_simulator.controllers.MultiAgentPlanner.MultiAgentPlanner",
    "high_level_ctrl": "reactive_control",  # choose from hj_naive, flocking, reactive_control
    "unit": "km",
    "communication_thrsld": 9,
    "hj_specific_settings": HJMultiTimeConfig,
    "flocking_config": flockingConfig,
    "reactive_control_config": reactiveConfig,
}

In [ ]:
exp.experimentName = exp.get("experiment_name")
# add controller
exp.addController(name="reactive_ctrl", ctrlConfig=MultiAgentCtrlConfig)

#### Add Flocking Controller

In [ ]:
MultiAgentCtrlConfig = {
    "ctrl_name": "ocean_navigation_simulator.controllers.MultiAgentPlanner.MultiAgentPlanner",
    "high_level_ctrl": "flocking",  # choose from hj_naive, flocking, reactive_control
    "unit": "km",
    "communication_thrsld": 9,
    "hj_specific_settings": HJMultiTimeConfig,
    "flocking_config": flockingConfig,
    "reactive_control_config": reactiveConfig,
}

In [ ]:
exp.experimentName = exp.get("experiment_name")
exp.addController(name="flocking", ctrlConfig=MultiAgentCtrlConfig)

# Set missions to ready to run

In [ ]:
experiment = "experiment.id==" + '"' + exp.id + '"'
status = " && status==" + '"staged_for_feasibility"'
exp_list = c3.Mission.fetch(spec={"include": "[this]", "filter": experiment}).objs
print(len(exp_list))

In [ ]:
for miss in exp_list:
    miss.status = "ready_to_run"
    miss.merge()

# Create Ocean Sim Runs

In [ ]:
# Create the Run objects
exp.createOceanSimRuns(ctrl_name="flocking", obs_name="NoObserver")

## Run simulation debug 

In [ ]:
exp_filter = "mission.experiment.id==" + '"' + exp.get("experimentName").experimentName + '"'
runs = c3.OceanSimRun.fetch(
    spec={"include": "[this]", "filter": exp_filter, "order": "ascending(mission.id)", "limit": -1}
).objs

In [ ]:
run = runs[-2]

In [ ]:
run

In [ ]:
# ensure we have all necessary data to run
this = runs[-2].get(
    "mission.missionConfig, mission.experiment.timeout_in_sec,"
    + "mission.experiment.arenaConfig, mission.experiment.objectiveConfig,"
    + "controllerSetting.ctrlConfig, observerSetting.observerConfig"
)
this

In [ ]:
# update the entry while it's running
new_osr = c3.OceanSimRun(**{"id": this.id, "status": "running_sim"})
new_osr.merge()

In [ ]:
import os
import pickle
import logging
import datetime
import numpy as np
import wandb
import matplotlib.pyplot as plt
import pandas as pd

import ocean_navigation_simulator
from ocean_navigation_simulator.environment.ArenaFactory import ArenaFactory
from ocean_navigation_simulator.problem_factories.Constructor import Constructor
from ocean_navigation_simulator.environment.PlatformState import SpatioTemporalPoint

In [ ]:
# create strings for all files and external directories where to save results
# Set up file paths and download folders
temp_folder = "/tmp/" + this.id + "/"
# create the folder if it doesn't exist yet
if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)
log_file_name = this.id + ".log"
logging.basicConfig(filename=temp_folder + log_file_name, level=logging.DEBUG)
traj_file_name = this.id + ".obj"
extDir = "ocean_sim_run_results/" + this.mission.experiment.id + "/OceanSimRuns/" + this.id

# set download directories (ignore set ones in arenaConfig)
arenaConfig = this.mission.experiment.arenaConfig

# Get and set the correct path to the nutrient and monthly average files (for c3) - !! CONFIG gets overwritten!!
filepath = ocean_navigation_simulator.__file__
module_path = os.path.dirname(filepath)
results_path = module_path + "/results/"
# nutrient_path = module_path + "/package_data/nutrients/"
# averages_path = module_path + "/package_data/monthly_averages/"

# arenaConfig["seaweed_dict"]["hindcast"]["source_settings"][
#     "filepath"
# ] = nutrient_path

arenaConfig["timeout"] = this.mission.experiment.timeout_in_sec
to_download_forecast_files = False

# for hindcast
arenaConfig["ocean_dict"]["hindcast"]["source_settings"]["folder"] = (
    "/tmp/" + this.id + "/hindcast_files/"
)
# for forecast
if arenaConfig["ocean_dict"]["forecast"] is not None:
    arenaConfig["ocean_dict"]["forecast"]["source_settings"]["folder"] = (
        "/tmp/" + this.id + "/forecast_files/"
    )
    to_download_forecast_files = arenaConfig["ocean_dict"]["forecast"]["source"] == "forecast_files"

# prepping the file download
point_to_check = SpatioTemporalPoint.from_dict(this.mission.missionConfig["x_0"][0])
t_interval = [
    point_to_check.date_time,
    point_to_check.date_time
    + datetime.timedelta(
        seconds=this.mission.experiment.timeout_in_sec
        + arenaConfig["casadi_cache_dict"]["time_around_x_t"]
        + 7200
    ),
]

arena, controller = None, None

import contextlib


@contextlib.contextmanager
def dummy_context_mgr():
    yield None

In [ ]:
with (
    ArenaFactory.download_files(
        config=arenaConfig,
        type="hindcast",
        t_interval=t_interval,
        c3=c3,
        points=[point_to_check.to_spatial_point()],
    ) as download_hindcast_files_to_local,
    ArenaFactory.download_files(
        config=arenaConfig,
        type="forecast",
        t_interval=t_interval,
        c3=c3,
        points=[point_to_check.to_spatial_point()],
    )
    if to_download_forecast_files
    else dummy_context_mgr() as download_forecast_files_to_local,
):
    # Step 0: Create Constructor object which contains arena, problem, controller and observer
    constructor = Constructor(
        arena_conf=arenaConfig,
        mission_conf=this.mission.missionConfig,
        objective_conf=this.mission.experiment.objectiveConfig,
        ctrl_conf=this.controllerSetting.ctrlConfig,
        observer_conf=this.observerSetting.observerConfig,
    )

    # Step 1.1 Retrieve problem
    problem = constructor.problem

    # Step 1.2: Retrieve arena
    arena = constructor.arena
    observation = arena.reset(platform_set=problem.start_state)
    problem_status = arena.problem_status(problem=problem)

    # Step 2: Retrieve Controller
    controller = constructor.controller

    # log initial reachability snapshot as .jpg and .svg
    controller.get_action(observation=observation)  # needed for init of controller
    ax = controller.plot_reachability_snapshot(
        rel_time_in_seconds=0,
        granularity_in_h=5,
        alpha_color=1,
        time_to_reach=True,
        fig_size_inches=(12, 12),
        plot_in_h=True,
        return_ax=True,
    )
    fig = ax.get_figure()
    fig = plt.figure(fig)
    plt.savefig(temp_folder + "reachability_snapshot.svg")
    plt.savefig(temp_folder + "reachability_snapshot.jpg", dpi=80)

    # Step 3: Retrieve observer
    observer = constructor.observer
    observer.observe(observation)
    observation.forecast_data_source = observer

    # Step 4: Run Arena
    ctrl_deviation_from_opt = []
    all_pltf_status = [0] * len(this.mission.missionConfig["x_0"])
    min_distances_to_target_over_mission = [np.inf] * len(this.mission.missionConfig["x_0"])
    pb_timeout_flag = -1

In [ ]:
while not any(status == pb_timeout_flag for status in problem_status):
    # Get action
    action, ctrl_correction = controller.get_action(observation=observation)
    ctrl_deviation_from_opt.append(ctrl_correction)
    # execute action
    observation = arena.step(action)

    # collect data
    min_distances_to_target_over_mission = arena.get_min_or_max_of_two_lists(
        list_a=min_distances_to_target_over_mission,
        list_b=arena.final_distance_to_target(problem=problem),
        min_or_max="min",
    )
    # Observer data assimilation
    observer.observe(observation)
    observation.forecast_data_source = observer

    # update problem status
    problem_status = arena.problem_status(problem=problem)
    # for the final metric, look if platform was able to reach target within T,
    # so keep only max (=1 if pltf reached target)
    all_pltf_status = arena.get_min_or_max_of_two_lists(
        list_a=all_pltf_status, list_b=problem_status, min_or_max="max"
    )

In [ ]:
new_osr.status = "finished_running"
new_osr.terminationReason = "Timeout"  # arena.problem_status_text(problem_status)
logged_error_message = None

In [ ]:
# Step 4: create the OceanSimResult object with the files and upload it
if arena is not None:
    # Create a large dict with all trajectory data
    trajs_dict = {
        "sim_traj": arena.state_trajectory,
        "sim_ctrl": arena.action_trajectory,
    }
    # if controller is not None:
    #     if len(controller.planned_trajs) > 0:
    #         trajs_dict["plans"] = controller.planned_trajs

    with open(temp_folder + traj_file_name, "wb") as traj_file:
        pickle.dump(trajs_dict, traj_file)

    # Step 4.3 upload the traj_file_name together with the log_file_name to blob storage
    # upload log file
    new_osr.oceanSimResult = c3.OceanSimResult()
    if os.path.exists(temp_folder + log_file_name):
        c3.Client.uploadLocalClientFiles(
            temp_folder + log_file_name,
            extDir + "/logs",
            {"peekForMetadata": True},
        )
        log_file = c3.File(**{"url": extDir + "/logs/" + log_file_name}).readMetadata()
        new_osr.oceanSimResult.log_file = log_file
        os.remove(temp_folder + log_file_name)

    if os.path.exists(temp_folder + traj_file_name):
        # upload traj file
        c3.Client.uploadLocalClientFiles(
            temp_folder + traj_file_name,
            extDir + "/trajs",
            {"peekForMetadata": True},
        )
        trajs_file = c3.File(**{"url": extDir + "/trajs/" + traj_file_name}).readMetadata()
        new_osr.oceanSimResult.trajectories = trajs_file
        os.remove(temp_folder + traj_file_name)

    # # log shortest distance to target
    # if this.mission.experiment.objectiveConfig["type"] == "nav":
    #     new_osr.minDistanceToTarget = arena.final_distance_to_target(
    #         problem=problem
    #     ) # we already log this in the metric and here we have a list of minDistances instead of a just a float

### Prepare data for WandB
ctrlConfig = this.controllerSetting.ctrlConfig
missionConfig = this.mission.missionConfig
observerConfig = this.observerSetting.observerConfig

# Prepare variables for run naming

# Planning horizon in days
# T_in_days = ctrlConfig["hj_specific_setting"]["T_goal_in_seconds"] / (24 * 3600)

# Prepare string whether we use only HC or FC/HC
if arenaConfig["ocean_dict"]["forecast"] is not None:
    data_sources = "FC_HC"
else:
    data_sources = "HC"

region = arenaConfig["ocean_dict"].get("region", "GOM")

nb_platforms = len(missionConfig["x_0"])
ctrl_name = ctrlConfig["high_level_ctrl"]

umax = arenaConfig["platform_dict"]["u_max_in_mps"]

# Log metrics in WandB
os.environ["WANDB_API_KEY"] = "1f19232e6ccc9981a8a972bee18ba31a94644835"

wandb.init(
    # Set the project where this run will be logged
    project="Master Thesis",
    id=this.id,
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"{this.id}_nb_pltf_{nb_platforms}_{region}_{data_sources}",
    # Track hyperparameters and run metadata
    config={
        "missionConfig": missionConfig,
        "arenaConfig": arenaConfig,
        "ctrlConfig": ctrlConfig,
        "observerConfig": observerConfig,
        "mission.id": this.mission.id,
        "experiment.id": this.mission.experiment.id,
    },
    entity="nhoischen",
)
metrics_dict = arena.save_metrics_to_log(
    all_pltf_status=all_pltf_status,
    min_distances_to_target=min_distances_to_target_over_mission,
    max_correction_from_opt_ctrl=ctrl_deviation_from_opt,
    filename=f"{temp_folder}/metrics_{this.id}.log",
)
metrics_df = pd.DataFrame(data=metrics_dict, index=[0])
wandb.log({"metrics": metrics_df}, commit=False)

# Log state_trajectory as table/dataframe
wandb.log(
    {"data_to_plot": wandb.Table(dataframe=arena.get_plot_data_for_wandb())},
    commit=False,
)

# log trajectory plot on ocean current as .jpg and .svg
ax = arena.plot_all_on_map(
    problem=problem,
    show_control_trajectory=False,
    return_ax=True,
)
fig = ax.get_figure()
fig = plt.figure(fig)
plt.savefig(temp_folder + "current_trajectory_on_map.svg")
plt.savefig(temp_folder + "current_trajectory_on_map.jpg", dpi=80)

# log inter-platform distance evolution as .jpg and .svg
ax = arena.plot_distance_evolution_between_platforms(
    xticks_temporal_res=arenaConfig["timeout"] / 12
)
fig = ax.get_figure()
fig = plt.figure(fig)
plt.savefig(temp_folder + "distance_evolution.svg")
plt.savefig(temp_folder + "distance_evolution.jpg", dpi=80)

# log images in wandb
image_traj = wandb.Image(
    temp_folder + "current_trajectory_on_map.jpg",
    caption="Platform trajectories on currents map",
)
image_reach = wandb.Image(
    temp_folder + "reachability_snapshot.jpg",
    caption="Initial reachability plot",
)

image_dist = wandb.Image(
    temp_folder + "distance_evolution.jpg",
    caption="Distance evolution between platform pairs",
)
wandb.log({"Inter-platform distance evolution": image_dist}, commit=False)
wandb.log({"Reachability snapshot": image_reach}, commit=False)
wandb.log({"Platform trajectories": image_traj}, commit=True)

# log trajectory animations
arena.animate_trajectory(
    margin=0.25,
    problem=problem,
    temporal_resolution=7200,
    background="current",
    output=f"{temp_folder}platform_trajectories.mp4",
    fps=10,
)
arena.animate_graph_net_trajectory(
    temporal_resolution=7200,
    plot_ax_ticks=True,
    output=f"{temp_folder}network_graph_trajectory.mp4",
    fps=5,
)

### Save files to c3 and add link to WandB
traj_currents_plot_file_name = "current_trajectory_on_map.svg"
reachability_plot_file_name = "reachability_snapshot.svg"
distance_evol_plot_file_name = "distance_evolution.svg"
traj_currents_animation_file_name = "platform_trajectories.mp4"
traj_network_animation_file_name = "network_graph_trajectory.mp4"

# add platform trajectories plot
c3.Client.uploadLocalClientFiles(
    temp_folder + traj_currents_plot_file_name,
    extDir + "/logs",
    {"peekForMetadata": True},
)
traj_currents_plot_file = c3.File(
    **{"url": extDir + "/logs/" + traj_currents_plot_file_name}
).readMetadata()
new_osr.oceanSimResult.traj_currents_plot_file = traj_currents_plot_file
os.remove(temp_folder + traj_currents_plot_file_name)

# add reachability plot
c3.Client.uploadLocalClientFiles(
    temp_folder + reachability_plot_file_name,
    extDir + "/logs",
    {"peekForMetadata": True},
)
reachability_plot_file = c3.File(
    **{"url": extDir + "/logs/" + reachability_plot_file_name}
).readMetadata()
new_osr.oceanSimResult.reachability_plot_file = reachability_plot_file
os.remove(temp_folder + reachability_plot_file_name)

# add distance evolution plot
c3.Client.uploadLocalClientFiles(
    temp_folder + distance_evol_plot_file_name,
    extDir + "/logs",
    {"peekForMetadata": True},
)
distance_evol_plot_file = c3.File(
    **{"url": extDir + "/logs/" + distance_evol_plot_file_name}
).readMetadata()
new_osr.oceanSimResult.distance_evol_plot_file = distance_evol_plot_file
os.remove(temp_folder + distance_evol_plot_file_name)

# add platform trajectories animation
c3.Client.uploadLocalClientFiles(
    temp_folder + traj_currents_animation_file_name,
    extDir + "/logs",
    {"peekForMetadata": True},
)
traj_currents_animation_file = c3.File(
    **{"url": extDir + "/logs/" + traj_currents_animation_file_name}
).readMetadata()
new_osr.oceanSimResult.traj_currents_animation_file = traj_currents_animation_file
os.remove(temp_folder + traj_currents_animation_file_name)

# add graph animation
c3.Client.uploadLocalClientFiles(
    temp_folder + traj_network_animation_file_name,
    extDir + "/logs",
    {"peekForMetadata": True},
)
traj_network_animation_file = c3.File(
    **{"url": extDir + "/logs/" + traj_network_animation_file_name}
).readMetadata()
new_osr.oceanSimResult.traj_network_animation_file = traj_network_animation_file
os.remove(temp_folder + traj_network_animation_file_name)

# LOG URL IN WANDB
wandb.summary["traj_current_plot_svg"] = traj_currents_plot_file.generatePresignedUrl("GET", "600d")
wandb.summary["reachability_plot_svg"] = reachability_plot_file.generatePresignedUrl("GET", "600d")
wandb.summary["distance_evol_plot_svg"] = distance_evol_plot_file.generatePresignedUrl(
    "GET", "600d"
)
wandb.summary["platf_trajectories_animation"] = traj_currents_animation_file.generatePresignedUrl(
    "GET", "600d"
)
wandb.summary["traj_network_animation"] = traj_network_animation_file.generatePresignedUrl(
    "GET", "600d"
)

wandb.finish()

In [ ]:
ax = controller.plot_reachability_snapshot(
    rel_time_in_seconds=0,
    granularity_in_h=5,
    alpha_color=1,
    time_to_reach=True,
    fig_size_inches=(12, 12),
    plot_in_h=True,
    return_ax=True,
)

In [ ]:
wandb.finish()

In [ ]:
ax = arena.plot_all_on_map(
    problem=problem,
    show_control_trajectory=False,
    return_ax=True,
)

In [ ]:
arena.animate_trajectory(
    margin=0.25,
    problem=problem,
    temporal_resolution=7200,
    background="current",
    output=f"{temp_folder}platform_trajectories.mp4",
    fps=10,
)

# Run Simulation

In [ ]:
# Create the Experiment Batch Job Options
options = c3.OceanSimRunBatchJobOptions(
    **{
        "experiment": exp,
        "actionRuntime": "andreas",  # enum('marius', 'andreas', 'matthias', 'nicolas')
    }
)

In [ ]:
# Start the Batch Job
run_job = c3.OceanSimRunBatchJob(**{"options": options.toJson()}).upsert()
# run_job.numAutoRecoveryAttempts = 1
run_job.start()
run_job

In [ ]:
# run_job = c3.OceanSimRunBatchJob.get('03b44bae-1a6e-4fa1-b07e-d906b48b84bf')
run_job.status()

In [ ]:
experiment = "mission.experiment.id==" + '"' + exp.id + '"'
objs_list = c3.OceanSimRun.fetch(spec={"include": "[this]", "filter": experiment}).objs
print("Total of {} OceanSimRuns".format(len(objs_list)))

staged_to_run = 0
running_sim = 0
finished_running = 0
run_failed = 0
for run in objs_list:
    if run.status == "staged_to_run":
        staged_to_run += 1
    elif run.status == "running_sim":
        running_sim += 1
    elif run.status == "finished_running":
        finished_running += 1
    elif run.status == "run_failed":
        run_failed += 1

print("staged_to_run: ", staged_to_run)
print("running_sim: ", running_sim)
print("finished_running: ", finished_running)
print("run_failed: ", run_failed)

In [ ]:
c3.Cluster.actionDump()

In [ ]:
exp_filter = "mission.experiment.id==" + '"' + exp.get("experimentName").experimentName + '"'
all_exp_runs = c3.OceanSimRun.fetch(
    spec={"include": "[this]", "filter": exp_filter, "order": "ascending(mission.id)", "limit": -1}
).objs

In [ ]:
all_exp_runs

In [ ]:
import pandas as pd


def get_results_df(this):
    # get all runs from the database (NOTE this can only get 2000 at a time!)
    exp_filter = "mission.experiment.id==" + '"' + this.get("experimentName").experimentName + '"'
    all_exp_runs = c3.OceanSimRun.fetch(
        spec={
            "include": "[this]",
            "filter": exp_filter,
            "order": "ascending(mission.id)",
            "limit": -1,
        }
    ).objs
    # get a list of all termination reasons
    termination_reason = []
    status = []
    arrival_time = []
    minDistanceToTarget = []
    ctrl_obs_name = []
    mission_ids = []

    for run in all_exp_runs:
        # get mission_id
        mission_ids.append(run.get("mission").mission.id)
        # get ctrl_obs setting
        ctrl_obs_name.append(
            run.get("controllerSetting.name").controllerSetting.name
            + "__"
            + run.get("observerSetting.name").observerSetting.name
        )
        # add outcomes
        termination_reason.append(run.terminationReason)
        status.append(run.status)
        arrival_time.append(run.T_arrival_time)
        minDistanceToTarget.append(run.minDistanceToTarget)

    # Make a df out of it
    df = pd.DataFrame(
        list(
            zip(
                mission_ids,
                ctrl_obs_name,
                status,
                termination_reason,
                arrival_time,
                minDistanceToTarget,
            )
        ),
        columns=[
            "mission_id",
            "setting",
            "status",
            "termination_reason",
            "arrival_time",
            "minDistance",
        ],
    )

    return df

In [ ]:
df = get_results_df(exp)

In [ ]:
df